<a href="https://colab.research.google.com/github/dannygallagher/CIS519-Project/blob/main/Baseline_Char_Bigram_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import gzip
import json
import json
import gzip
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.feature_extraction.text import TfidfTransformer
def get_metrics(true_labels, pred_labels):
    f1 = f1_score(true_labels, pred_labels, average='macro')
    acc = accuracy_score(true_labels, pred_labels)
    rec = recall_score(true_labels, pred_labels, average='macro')
    prec = precision_score(true_labels, pred_labels, average='macro')
    return f1, acc, rec, prec

def fast_parse(path):
  # path = path + ".json.gz"
  max_reviews = 1689188
  iter = 5000
  counter = 0 
  all_reviews = []
  g = open(path, 'r')
  for line in g:
    d = json.loads(line)
    all_reviews.append(d)
    if (counter % iter) == 0:
      print(f"On line{counter}, {counter/max_reviews} % done")
    counter = counter + 1
  return all_reviews

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
total_results = pd.DataFrame(columns=col_list)

All 5 Cat

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Char Bigram/results_All_5Cat.csv'
# df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.json')
# df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.json')
# df_experiment = pd.from_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/elec_test_5class.csv')
# df_train = pd.DataFrame().from_dict(df_train)
# df_test = pd.DataFrame().from_dict(df_test)

df_train = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.csv')
df_exp = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test.csv')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (2,2),tokenizer = token.tokenize, analyzer="char")

tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['overall']
y_test = df_test['overall']
y_exp = df_exp['overall']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)

from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = MNB.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["5Cat", "Char Bigram", "All", "All-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = MNB.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["5Cat", "Char Bigram", "All", "All-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = MNB.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["5Cat", "Char Bigram", "All", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)

48.65%
37.53%
43.98%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Char Bigram,All,All-Train,0.486477,0.483844,0.483874,0.495822
0,5Cat,Char Bigram,All,All-Test,0.375273,0.501692,0.487216,0.370072
0,5Cat,Char Bigram,All,Elec-Test,0.439754,0.449353,0.452651,0.474091


Elec 5 Cat

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Char Bigram/results_Elec_5Cat.csv'
# df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.json')
# df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.json')
# df_experiment = pd.from_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/elec_test_5class.csv')
# df_train = pd.DataFrame().from_dict(df_train)
# df_test = pd.DataFrame().from_dict(df_test)

df_train = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_train.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test.csv')
df_exp = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.csv')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (2,2),tokenizer = token.tokenize, analyzer="char")
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['overall']
y_test = df_test['overall']
y_exp = df_exp['overall']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)

from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = MNB.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["5Cat", "Char Bigram", "Elec", "Elec-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = MNB.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["5Cat", "Char Bigram", "Elec", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = MNB.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["5Cat", "Char Bigram", "Elec", "ALL-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


48.09%
46.12%
35.75%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Char Bigram,Elec,Elec-Train,0.480870,0.478117,0.478276,0.492024
0,5Cat,Char Bigram,Elec,Elec-Test,0.461161,0.464928,0.467897,0.477951
0,5Cat,Char Bigram,Elec,ALL-Test,0.357475,0.483912,0.462223,0.358774


ALL Binary

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Char Bigram/results_All_bin.csv'
df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train_binaryposneg.json')
df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test_binaryposneg.json')
df_exp = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test_binaryposneg.json')
df_train = pd.DataFrame().from_dict(df_train)
df_test = pd.DataFrame().from_dict(df_test)
df_exp = pd.DataFrame().from_dict(df_exp)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (2,2),tokenizer = token.tokenize, analyzer="char")
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['pos_neg']
y_test = df_test['pos_neg']
y_exp = df_exp['pos_neg']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)

from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(X_train, y_train)

On line0, 0.0 % done
On line5000, 0.002960002083841467 % done
On line10000, 0.005920004167682934 % done
On line15000, 0.008880006251524401 % done
On line20000, 0.011840008335365868 % done
On line25000, 0.014800010419207334 % done
On line30000, 0.017760012503048803 % done
On line35000, 0.02072001458689027 % done
On line40000, 0.023680016670731736 % done
On line45000, 0.026640018754573204 % done
On line50000, 0.02960002083841467 % done
On line55000, 0.03256002292225614 % done
On line60000, 0.035520025006097605 % done
On line65000, 0.03848002708993907 % done
On line70000, 0.04144002917378054 % done
On line75000, 0.04440003125762201 % done
On line80000, 0.04736003334146347 % done
On line85000, 0.050320035425304936 % done
On line90000, 0.05328003750914641 % done
On line95000, 0.05624003959298787 % done
On line100000, 0.05920004167682934 % done
On line105000, 0.06216004376067081 % done
On line110000, 0.06512004584451228 % done
On line115000, 0.06808004792835375 % done
On line120000, 0.071040

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = MNB.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["Binary", "Char Bigram", "All", "All-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = MNB.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["Binary", "Char Bigram", "All", "All-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = MNB.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["Binary", "Char Bigram", "All", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


76.75%
60.28%
70.69%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,Binary,Char Bigram,All,All-Train,0.767532,0.770206,0.770262,0.783366
0,Binary,Char Bigram,All,All-Test,0.602826,0.728548,0.798167,0.612456
0,Binary,Char Bigram,All,Elec-Test,0.706950,0.706950,0.746110,0.745967


Elec Bin

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Char Bigram/results_Elec_Bin.csv'
df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_train_binaryposneg.json')
df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test_binaryposneg.json')
df_exp = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test_binaryposneg.json')
df_train = pd.DataFrame().from_dict(df_train)
df_test = pd.DataFrame().from_dict(df_test)
df_exp = pd.DataFrame().from_dict(df_exp)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (2,2),tokenizer = token.tokenize, analyzer="char")
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['pos_neg']
y_test = df_test['pos_neg']
y_exp = df_exp['pos_neg']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)

from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(X_train, y_train)

On line0, 0.0 % done
On line5000, 0.002960002083841467 % done
On line10000, 0.005920004167682934 % done
On line15000, 0.008880006251524401 % done
On line20000, 0.011840008335365868 % done
On line25000, 0.014800010419207334 % done
On line30000, 0.017760012503048803 % done
On line35000, 0.02072001458689027 % done
On line40000, 0.023680016670731736 % done
On line45000, 0.026640018754573204 % done
On line50000, 0.02960002083841467 % done
On line55000, 0.03256002292225614 % done
On line60000, 0.035520025006097605 % done
On line65000, 0.03848002708993907 % done
On line70000, 0.04144002917378054 % done
On line75000, 0.04440003125762201 % done
On line80000, 0.04736003334146347 % done
On line85000, 0.050320035425304936 % done
On line90000, 0.05328003750914641 % done
On line95000, 0.05624003959298787 % done
On line100000, 0.05920004167682934 % done
On line105000, 0.06216004376067081 % done
On line110000, 0.06512004584451228 % done
On line115000, 0.06808004792835375 % done
On line120000, 0.071040

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = MNB.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["Binary", "Char Bigram", "Elec", "Elec-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = MNB.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["Binary", "Char Bigram", "Elec", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = MNB.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["Binary", "Char Bigram", "Elec", "ALL-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


76.74%
74.23%
58.33%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,Binary,Char Bigram,Elec,Elec-Train,0.767414,0.770268,0.770680,0.785133
0,Binary,Char Bigram,Elec,Elec-Test,0.742268,0.743225,0.769788,0.758574
0,Binary,Char Bigram,Elec,ALL-Test,0.583316,0.708550,0.773758,0.601319


In [ ]:
total_results.to_csv('/content/drive/MyDrive/cis519_final/Baselines/Char Bigram/total_results.csv')

In [ ]:
total_results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Char Bigram,All,All-Train,0.486477,0.483844,0.483874,0.495822
0,5Cat,Char Bigram,All,All-Test,0.375273,0.501692,0.487216,0.370072
0,5Cat,Char Bigram,All,Elec-Test,0.439754,0.449353,0.452651,0.474091
0,5Cat,Char Bigram,Elec,Elec-Train,0.480870,0.478117,0.478276,0.492024
0,5Cat,Char Bigram,Elec,Elec-Test,0.461161,0.464928,0.467897,0.477951
0,5Cat,Char Bigram,Elec,ALL-Test,0.357475,0.483912,0.462223,0.358774
0,Binary,Char Bigram,All,All-Train,0.767532,0.770206,0.770262,0.783366
0,Binary,Char Bigram,All,All-Test,0.602826,0.728548,0.798167,0.612456
0,Binary,Char Bigram,All,Elec-Test,0.706950,0.706950,0.746110,0.745967
0,Binary,Char Bigram,Elec,Elec-Train,0.767414,0.770268,0.770680,0.785133
